In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# !pip install pdfplumber -q
# !apt-get install ocrmypdf -q   #installing OCR library 

In [ ]:
import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import BigramCollocationFinder
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import*
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
# with pdfplumber.open("/content/drive/MyDrive/test.pdf") as pdf:
#     text=""
#     pages = pdf.pages
#     for page in pages:
#         text += page.extract_text(x_tolerance=2)
#         print(text)

# Import CV and scraped data

In [ ]:
# Import CV
# f=open(r"C:\ikhsan\UNAIR\MBKM\Bangkit\Profile.pdf",'r', errors = 'ignore')
# text=f.read()

# import pdfminer
# from pdfminer.high_level import extract_text

# # Read PDF file and extract text
# def read_pdf(file_path):
#     text = extract_text(file_path)
#     # print(text)
#     return text

# # Preprocess text
# def preprocess_text(text):
#     # Implement text preprocessing steps as per your requirements
#     # For example, remove special characters, convert to lowercase, etc.
#     # print(text)
#     return text

# resume_path = r"C:\ikhsan\UNAIR\Template CV ATS-Friendly Glints\CV panjang.pdf"
# resume_text = read_pdf(resume_path)
# text = preprocess_text(resume_text)

with pdfplumber.open(r"C:\ikhsan\UNAIR\Template CV ATS-Friendly Glints\CV panjang.pdf") as pdf:
    text=""
    pages = pdf.pages
    for page in pages:
        text += page.extract_text(x_tolerance=2)
        print(text)

# Locations
# f1=open(r"data\Location.txt",'r', errors = 'ignore')
# text1=f1.read()
# locations = word_tokenize(text1.replace("\n", " "))

# Additional stopwords
f2=open(r"data\stopwords.txt",'r', errors = 'ignore')
text2=f2.read()
stopwords_additional = word_tokenize(text2.replace("\n", " "))

In [ ]:
def nlp(x):
    word_sent = word_tokenize(x.lower().replace("\n",""))
    _stopwords = set(stopwords.words('english', 'indonesian') + list(punctuation)+list("●")+list('–')+list('’')+stopwords_additional)
    word_sent=[word for word in word_sent if word not in _stopwords]
    lemmatizer = WordNetLemmatizer()
    NLP_Processed_CV = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(word_sent))]
#     return " ".join(NLP_Processed_CV)
    return NLP_Processed_CV

In [ ]:
def get_key_word(x):
    finder = BigramCollocationFinder.from_words(x)
    keyword = []
    keywords_CV = []
    for i in sorted(finder.ngram_fd.items()):
    # if a double word appears more than once, then print it out.
        if i[1] > 1:
            print(i)
            keyword.append(i[0])
        else:
            pass

    # print("************************")
    for j in keyword:
    #     print(" ".join(j))
        keywords_CV.append(" ".join(j))
    return keywords_CV

In [ ]:
NLP_Processed_CV = nlp(text)
# NLP_Processed_CV=' '.join(NLP_Processed_CV)
NLP_Processed_CV

In [ ]:
key_word = get_key_word(NLP_Processed_CV)
key_word

In [ ]:
#Import scraped data
df = pd.read_excel(r"data\data website indeed Jakarta (30 May 2023) last.xlsx", "Part-Time")
index = []
for i in range (0, len(df)):
    index.append(i)
df.insert(0,'JobID', index)
df['All'] = df['Posisi'] + ' ' + df['Perusahaan '] + ' ' + df['Lokasi'] + ' ' + df['Tipe Pekerjaan'] + ' ' + df['Kualifikasi'] + ' ' + df['Deskripsi Pekerjaan ']
df.rename(columns = {'Perusahaan ':'Perusahaan', 'Deskripsi Pekerjaan ': 'Deskripsi Pekerjaan'}, inplace = True)
df.head(2)

In [ ]:
# print(NLP_Processed_CV)
# print(key_word)
df.shape

In [ ]:
df.info()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
tfidf_jobid = tfidf_vectorizer.fit_transform(df['All'].apply(lambda x: np.str_(x)))
# print(tfidf_jobid)
user_tfidf = tfidf_vectorizer.transform(df['All'].apply(lambda x: np.str_(x)))
# print(user_tfidf)

In [ ]:
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)
outputJob = list(cos_similarity_tfidf)
outputJob

# Put keyword/CV into a dataframe

In [ ]:
df2 = pd.DataFrame()
# append columns to an empty DataFrame
df2['Posisi'] = ["I"]
#df2['job highlights'] = ["I"]
df2['Deskripsi Pekerjaan'] = ["I"]
# df2['company overview'] = ["I"]
df2['Perusahaan'] = ["I"]

# Compare with the key words from CV only
df2['All'] = " ".join(key_word)
df2.head()
# Compare with the entire CV
# df2['All'] = " ".join(NLP_Processed_CV)
# df2

# Create get_recommendation function

In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['JobID',  'Posisi', 'Perusahaan', 'Deskripsi Pekerjaan','Score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        recommendation.at[count, 'JobID'] = df.index[i]
        recommendation.at[count, 'Posisi'] = df['Posisi'][i]
        recommendation.at[count, 'Perusahaan'] = df['Perusahaan'][i]
#         recommendation.at[count, 'location'] = df['location'][i]
        recommendation.at[count, 'Deskripsi Pekerjaan'] = df['Deskripsi Pekerjaan'][i]
        recommendation.at[count, 'Score'] =  scores[count]
        count += 1
    return recommendation

# TFIDF

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x)))
    # tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv.apply(lambda x: np.str_(x)))
    # user_tfidf = tfidf_vectorizer.transform(cv)

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    output2 = list(cos_similarity_tfidf)
    return output2

outputTF = TFIDF(df['All'], df2['All'])
outputTF

In [ ]:
top = sorted(range(len(outputTF)), key=lambda i: outputTF[i], reverse=True)[:100]
list_scores = [outputTF[i][0][0] for i in top]
TF = get_recommendation(top, df, list_scores)
TF

# Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# print(tfidf_jobid)
user_tfidf = tfidf_vectorizer.transform(df['All'].apply(lambda user_tfidf: np.str_(user_tfidf)))

def count_vectorize(scraped_data, cv):
    # CountV the scraped data
    count_vectorizer = CountVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
    # count_jobid = count_vectorizer.fit_transform(scraped_data) #fitting and transforming the vector
    count_jobid = count_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x)))

    # CountV the cv
    # user_count = count_vectorizer.transform(cv)
    user_count = count_vectorizer.transform(cv.apply(lambda x: np.str_(x)))
    cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
    output3 = list(cos_similarity_countv)
    return output3

outputCV = count_vectorize(df['All'], df2['All'])
outputCV

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
count_jobid = count_vectorizer.fit_transform(df['All'].apply(lambda x: np.str_(x))) #fitting and transforming the vector
print(count_jobid)


In [ ]:
user_count = count_vectorizer.transform(df2['All'].apply(lambda x: np.str_(x)))
print(user_count)

In [ ]:
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
outputCV = list(cos_similarity_countv)
outputCV

In [ ]:
top = sorted(range(len(outputCV)), key=lambda i: outputCV[i], reverse=True)[:100]
list_scores = [outputCV[i][0][0] for i in top]
cv = get_recommendation(top, df, list_scores)
cv

# KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
    n_neighbors = 100
    KNN = NearestNeighbors(n_neighbors=10)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x))))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv.apply(lambda x: np.str_(x))))
    top = NNs[1][0][0:]
    print(top)
    index_score = NNs[0][0][0:]

    knn = get_recommendation(top, df, index_score)
    return knn

knn = KNN(df['All'], df2['All'])
knn

In [ ]:
# knn.to_csv('knn.csv', index=False)

# Combined TFIDF, CV and KNN result together, to make a dataframe "final"

In [ ]:
# merge1 = knn[['JobID','title', 'score']].merge(TF[['JobID','score']], on= "JobID")
# final = merge1.merge(cv[['JobID','score']], on = "JobID")
# final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
# final.head()

merge1 = knn[['JobID','Posisi', 'Score']].merge(TF[['JobID','Score']], on= "JobID")
final = merge1.merge(cv[['JobID','Score']], on = "JobID")
final = final.rename(columns={"Score_x": "KNN", "Score_y": "TF-IDF","Score": "CV"})
final.head()

In [ ]:
# reverseScoring(knn, high, 'score').to_csv('test.csv', index=False)

# Scale and assign weights

In [ ]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

In [ ]:
# final.to_csv('final.csv', index=False)
# final2 = final.sort_values(by="Final", ascending=False).copy()
# final2.head(10)

In [ ]:
# final2.merge(df, on="JobID")
final_all = df.merge(final, on="JobID")
final_job = final_all.sort_values(by="Final", ascending=False).copy()
final_job

In [ ]:
import pickle
pickle.dump(TF, open('artifacts/TF.pkl', 'wb'))
pickle.dump(cv, open('artifacts/cv.pkl', 'wb'))
pickle.dump(knn, open('artifacts/knn.pkl', 'wb'))
pickle.dump(final, open('artifacts/final.pkl', 'wb'))
pickle.dump(final_job, open('artifacts/final_job.pkl', 'wb'))